In [1]:
import numpy as np
import os
os.environ['NUMBA_THREADING_LAYER'] = 'omp' # my TBB version complains
os.environ['NUMBA_NUM_THREADS'] = '2'
os.environ['NUMBA_ENABLE_AVX'] = '1'
os.environ['NUMBA_CPU_NAME'] = 'generic'
os.environ['NUMBA_CPU_FEATURES'] = '+sse,+sse2,+avx,+avx2,+avx512f,+avx512dq'
#os.environ['NUMBA_CPU_FEATURES'] = '+sse,+sse2,+avx,+avx2'
import sys
sys.path.insert(0,'/Users/ban115/bolton/craco-python/src/')


import numba
from numba import njit, prange
from pylab import *
%matplotlib notebook

%load_ext autoreload
%autoreload 2


In [2]:
nant = 30
nbl = nant*(nant+1)//2
nt = 32
nchan = 24
nbeam = 36

vin = np.arange(nbl*nt*nchan*2, dtype=np.int16).reshape((nchan, nbl, nt, 2))
vavg = np.zeros((nchan, nbl, 2), dtype=np.float32)
ics = np.zeros((nt, nchan), dtype=np.float32)
cas = np.zeros((nt, nchan), dtype=np.float32)

vin.size

714240

In [3]:
# For type hint info see https://numba.pydata.org/numba-doc/0.12.2/tutorial_types.html
@njit(fastmath=True, parallel=True, locals={'vabs':numba.float32})
def make_averages(nant, vin, vavg, ics, cas):
    nchan,nbl, nt, _ = vin.shape
    
    for ic in range(nchan):
        ibl = 0
        for ia1 in range(nant):
            for ia2 in range(ia1, nant):
                #print(ia1, ia2, ibl, nbl)
                #assert ibl < nbl#, f'Invalid ibl {ia1} {ia2} {ibl} {nbl}'
                
                for it in range(nt):
                    v = vin[ic, ibl, it, :]
                    #v = np.array([0,1])
                    vabs = v[0]*v[0] + v[1]*v[1]
                    if ia1 == ia2:
                        ics[it, ic] += vabs
                    else:
                        cas[it, ic] += vabs

                    #print('out', ibl, ic, it)
                    vavg[ic, ibl, 0] += v[0]
                    vavg[ic, ibl, 1] += v[1]
                    
                    #print('done', ibl, ic, it)


                ibl += 1

    ics //= nant
    cas //= nbl
    vavg //= nt
            


In [4]:
for i in range(1):
    make_averages(nant, vin, vavg, ics, cas)

/data/craco/ban115/craco-python/venv310/lib/python3.10/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../tmp/ipykernel_26954/3996752812.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


In [5]:
make_averages.parallel_diagnostics(level=4)


 
 Parallel Accelerator Optimizing:  Function make_averages, 
/tmp/ipykernel_26954/3996752812.py (2)  
No source available
--------------------------------- Fusing loops ---------------------------------
Attempting fusion of parallel loops (combines loops with similar properties)...
----------------------------- Before Optimisation ------------------------------
--------------------------------------------------------------------------------
------------------------------ After Optimisation ------------------------------
Parallel structure is already optimal.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 
---------------------------Loop invariant code motion---------------------------
Allocation hoisting:
No allocation hoisting found

Instruction hoisting:
No instruction hoisting found
--------------------------------------------------------------------------------


In [6]:
%timeit -n1 make_averages(nant, vin, vavg, ics, cas)

1.21 ms ± 81.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
vin.shape

(24, 465, 32, 2)

In [8]:
make_averages.inspect_types()

make_averages (int64, Array(int16, 4, 'C', False, aligned=True), Array(float32, 3, 'C', False, aligned=True), Array(float32, 2, 'C', False, aligned=True), Array(float32, 2, 'C', False, aligned=True))
--------------------------------------------------------------------------------
# File: /tmp/ipykernel_26954/3996752812.py
# --- LINE 2 --- 
# label 0
#   nant = arg(0, name=nant)  :: int64
#   vin = arg(1, name=vin)  :: array(int16, 4d, C)
#   vavg = arg(2, name=vavg)  :: array(float32, 3d, C)
#   ics = arg(3, name=ics)  :: array(float32, 2d, C)
#   cas = arg(4, name=cas)  :: array(float32, 2d, C)

@njit(fastmath=True, parallel=True, locals={'vabs':numba.float32})

# --- LINE 3 --- 

def make_averages(nant, vin, vavg, ics, cas):

    # --- LINE 4 --- 
    #   $4load_attr.1 = getattr(value=vin, attr=shape)  :: UniTuple(int64 x 4)
    #   $6unpack_sequence.6 = exhaust_iter(value=$4load_attr.1, count=4)  :: UniTuple(int64 x 4)
    #   del $4load_attr.1
    #   $nchan.30 = static_getitem(val

In [9]:
from craco.cardcapfile import  get_single_packet_dtype, NCHAN,get_indexes
%aimport craco.card_averager
from craco.card_averager import *
import craco.card_averager as avg

In [10]:
nfpga = 6
nbeam = 36
nc_per_fpga = 4
nt = 16
tscrunch = 4

npkt = nbeam*nc_per_fpga
pktshape = (npkt, nt)
polsum = True
debughdr = True
dtype = get_single_packet_dtype(nbl, debughdr, polsum)
din_list = [np.zeros(pktshape, dtype=dtype) for i in range(nfpga)]
packets = [pkt for pkt in din_list]
packet_list = List()
[packet_list.append(self.dummy_packet if pkt is None else pkt) for pkt in packets]
_,_,auto_idxs,cross_idxs = get_indexes(nant)
din = np.array(din_list)
data = din['data']
print(din.shape)
print(data.shape)


(6, 144, 16)
(6, 144, 16, 2, 465, 1, 2)


In [11]:
packets[0]['data'].shape

(144, 16, 2, 465, 1, 2)

In [12]:
data.shape

(6, 144, 16, 2, 465, 1, 2)

In [13]:
(nfpga, nibc, nt1, nt2, nbl, npol, ncomp) = data.shape
data[:,:32*4,...].reshape(nfpga,NCHAN,32,nt1*nt2,nbl,npol,ncomp).transpose(2,1,0,3,4,5,6).reshape(32,nfpga*NCHAN,nt1*nt2,nbl,npol,ncomp).shape

(32, 24, 32, 465, 1, 2)

In [14]:
din['data'].shape

(6, 144, 16, 2, 465, 1, 2)

In [15]:
np.array?

Docstring:
array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
      like=None)

Create an array.

Parameters
----------
object : array_like
    An array, any object exposing the array interface, an object whose
    ``__array__`` method returns an array, or any (nested) sequence.
    If object is a scalar, a 0-dimensional array containing object is
    returned.
dtype : data-type, optional
    The desired data-type for the array. If not given, NumPy will try to use
    a default ``dtype`` that can represent the values (by applying promotion
    rules when necessary.)
copy : bool, optional
    If true (default), then the object is copied.  Otherwise, a copy will
    only be made if ``__array__`` returns a copy, if obj is a nested
    sequence, or if a copy is needed to satisfy any of the other
    requirements (``dtype``, ``order``, etc.).
order : {'K', 'A', 'C', 'F'}, optional
    Specify the memory layout of the array. If object is not an array, the
    newly crea

In [16]:
%timeit np.array(din_list)

12.8 ms ± 269 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit np.array(packets)

12.6 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit np.array(packet_list)

12.7 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
dout = avg.average0(din)

In [20]:
print(avg.average0(din).shape)
%timeit avg.average0(din)

(144, 16, 2, 465, 2)
46.4 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
print(avg.average1(din).shape)
%timeit avg.average1(din)

(144, 465, 2)
12.7 ms ± 470 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit dout.astype(np.int16)

1.64 ms ± 7.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
(nfpga, npkt, nt1, nt2, nbl, npol, _) = data.shape
# This reshape takes 19 milliseconds!
%timeit dreshape =  din['data'].reshape(nfpga, npkt, 1, nt1*nt2, nbl, npol, 2)


16.3 ms ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
print(avg.average2(din, tscrunch=32).shape)

%timeit avg.average2(din, tscrunch=4)

(144, 1, 465, 2)
28.5 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit average3(din, tscrunch=2)

29.3 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit average4(din, tscrunch=2)

28.1 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
data = din['data']
(nfpga, npkt, nt1, nt2, nbl, _, _) = data.shape
dshape = (npkt, nt1*nt2 // tscrunch, nbl, 2)
dout = np.zeros(dshape, dtype=np.float32)
%timeit avg.average4(din, tscrunch=tscrunch, dout=dout)

26.8 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit avg.average4(din, tscrunch=tscrunch, dout=None)

28.5 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
import llvmlite.binding as llvm
llvm.set_option('', '--debug-only=loop-vectorize')

In [30]:
data.dtype

dtype('int16')

In [31]:
dout.dtype

dtype('float32')

In [32]:
data.shape

(6, 144, 16, 2, 465, 1, 2)

In [33]:
_ = avg.average5(data, tscrunch=tscrunch, dout=dout)

/data/craco/ban115/craco-python/venv310/lib/python3.10/site-packages/numba/core/lowering.py:112: NumbaDebugInfoWarning: Could not find source for function: <function __numba_parfor_gufunc_0x7f4e78d35ed0 at 0x7f4e7827ff40>. Debug line information may be inaccurate.
  warnings.warn(NumbaDebugInfoWarning(msg))

LV: Checking a loop in "_ZN13_3cdynamic_3e36__numba_parfor_gufunc_0x7f4e78d35ed0B3v13B140c8tJTC_2fWQAliW1xhDEoY6EEMEUOEMISPGsAQMVj4QniQ4IXKQEMXwoMGLoQDDVsQR1NHAZtvoQrhyQ_2fKR8sTqKIYOQAmjYgkW7ADga6BRpMtejxVq4Ok1GuBzgLGOcSFAA_3d_3dE5ArrayIyLi1E1C7mutable7alignedE5ArrayIsLi3E1A7mutable7alignedE5ArrayIfLi3E1A7mutable7alignedE5ArrayIfLi3E1A7mutable7alignedE" from <string>:4:1
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B96
LV: Found an induction variable.
LV: Found an induction variable.
LV: Did not find one integer induction var.
LV: Can't vectorize due to memory conflicts
LV: Not vectorizing: Cannot prove legality.

LV: Checking a loop in "__gufunc__._ZN13_3cdynamic

In [34]:
%timeit avg.average5(data, tscrunch=tscrunch, dout=dout)

40.9 ms ± 7.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:

_ = average6(din, tscrunch=tscrunch, dout=dout)


/data/craco/ban115/craco-python/venv310/lib/python3.10/site-packages/numba/core/typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../src/craco/card_averager.py", line 309:
@njit(fastmath=True,debug=True,parallel=True)
def average6(din, tscrunch, dout):
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,

LV: Checking a loop in "_ZN5craco13card_averager8average6B3v14B124c8tJTC_2fWQAliW1xhDEoY6EEMEUOEMISPGsAQMVj4QniQ4IXKQEMXwoMGLoQDDVsQR1NHAZtvoQrhyQ_2fKR8sTqKIYOQBmNTRObNEjpVodJKVeC3QDMIIhzgEAE5ArrayI6RecordILi2031EELi3E1C7mutable9unalignedEx5ArrayIfLi4E1C7mutable7alignedE" from card_averager.py:312:1
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: .613.endif
LV: Not vectorizing: Control flow cannot be substituted for a s

In [36]:
tscrunch

4

In [37]:
%timeit average6(din, tscrunch=tscrunch, dout=dout)

17.8 ms ± 560 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
dout.shape

(144, 8, 465, 2)

In [39]:
average6.inspect_types()

average6 (Array(Record([('frame_id', {'type': uint64, 'offset': 0, 'alignment': None, 'title': None, }), ('bat', {'type': uint64, 'offset': 8, 'alignment': None, 'title': None, }), ('beam_number', {'type': uint8, 'offset': 16, 'alignment': None, 'title': None, }), ('sample_number', {'type': uint8, 'offset': 17, 'alignment': None, 'title': None, }), ('channel_number', {'type': uint8, 'offset': 18, 'alignment': None, 'title': None, }), ('fpga_id', {'type': uint8, 'offset': 19, 'alignment': None, 'title': None, }), ('nprod', {'type': uint16, 'offset': 20, 'alignment': None, 'title': None, }), ('flags', {'type': uint8, 'offset': 22, 'alignment': None, 'title': None, }), ('zero1', {'type': uint8, 'offset': 23, 'alignment': None, 'title': None, }), ('version', {'type': uint32, 'offset': 24, 'alignment': None, 'title': None, }), ('zero3', {'type': uint32, 'offset': 28, 'alignment': None, 'title': None, }), ('data', {'type': NestedArray(int16, (2, 465, 1, 2)), 'offset': 32, 'alignment': None, 

In [40]:
intermediate_dout = np.zeros(din['data'].shape, dtype=np.float32)
%timeit avg.average9(din, tscrunch=tscrunch, dout=intermediate_dout)

106 ms ± 724 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
dout.shape

(144, 8, 465, 2)

In [42]:
np.array(packets).shape

(6, 144, 16)

In [43]:
packets[0].dtype

dtype([('frame_id', '<u8'), ('bat', '<u8'), ('beam_number', 'u1'), ('sample_number', 'u1'), ('channel_number', 'u1'), ('fpga_id', 'u1'), ('nprod', '<u2'), ('flags', 'u1'), ('zero1', 'u1'), ('version', '<u4'), ('zero3', '<u4'), ('data', '<i2', (2, 465, 1, 2))])

In [44]:
packets[0].shape

(144, 16)

In [45]:
packets[0]['data'].shape

(144, 16, 2, 465, 1, 2)

In [46]:
(nibc, nt1, nt2, nbl, npol, _) = packets[0]['data'].shape
ntout = nt1 * nt2 // tscrunch
dout = np.zeros(packets[0]['data'].shape, dtype=np.float32)
dout1 = np.zeros((NBEAM*NCHAN,nt1*nt2//tscrunch,nbl,2), dtype=np.float32)

print(avg.average10(packets, dout=dout).shape)
%timeit avg.average10(packets, dout=dout)

(144, 16, 2, 465, 1, 2)
14.2 ms ± 52.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
%timeit dout.reshape(144,32//tscrunch,tscrunch,465,2).mean(axis=2)

2.04 ms ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%timeit dout.reshape(144,32//tscrunch,tscrunch,465,1,2).mean(axis=(2,4))

2.07 ms ± 37.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
%timeit dout1[:] = dout.reshape(144,32//tscrunch,tscrunch,465,1,2).mean(axis=(2,4))

2.74 ms ± 84.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
dout = np.zeros(packets[0]['data'].shape, dtype=np.float32)
print(avg.average11(packet_list,  dout=dout).shape)
%timeit avg.average11(packet_list,  dout=dout)

(144, 16, 2, 465, 1, 2)
55.2 ms ± 1.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
ntout

8

In [52]:
dout2 = avg.average10(packets, None)
print(avg.average13(packets, tscrunch, dout1, dout2).shape)
%timeit avg.average13(packets, tscrunch, dout1, dout2).shape

(144, 8, 465, 2)
18.5 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
vis_dout = np.zeros((NBEAM,len(cross_idxs),NCHAN,ntout,2), dtype=np.float32)
avg.vis_reshape(dout1, cross_idxs, vis_dout).shape
%timeit avg.vis_reshape(dout1, cross_idxs, vis_dout)

5.63 ms ± 91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
avg.average_pkts_and_reshape(packets, tscrunch, vis_dout, cross_idxs, dout1, dout2).shape
%timeit avg.average_pkts_and_reshape(packets, tscrunch, vis_dout, cross_idxs, d1=None, d2=None).shape

27.5 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# number is super slow even with normal numpy arrays

In [55]:
%timeit dout[:] = 0

1.19 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [56]:
%timeit dout * 3

2.4 ms ± 56.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
dout = np.zeros((144, 16, 2, 465, 1, 2), dtype=np.float32)

In [58]:
print(numba.__version__)

0.60.0


In [59]:
@njit
def numba_clear(d):
    d[:] = 0
    

In [60]:
%timeit dout[:]  = 0

1.24 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:
numba_clear(dout)
%timeit numba_clear(dout)


LV: Checking a loop in "_ZN8__main__11numba_clearB3v15B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dE5ArrayIfLi6E1C7mutable7alignedE" from numba_clear
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: .477.endif
LV: Not vectorizing: Control flow cannot be substituted for a select   br label %.501.if
LV: Interleaving disabled by the pass manager
LV: Can't if-convert the loop.
LV: Not vectorizing: Cannot prove legality.


2.43 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
dout_flat = dout.flatten()
numba_clear(dout_flat)
%timeit numba_clear(dout_flat)

1.97 ms ± 42.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
packets[0]['data'].reshape(144,32,465,2)

array([[[[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        ...,

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
      

# How about the card averager

In [64]:
from craco import card_averager

In [65]:
from numba.typed import List
data = List()
%timeit [data.append(self.dummy_packet if pkt is None else pkt) for pkt in packets]


14.2 µs ± 800 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [66]:
d = packets[0]
d['data'].shape

(144, 16, 2, 465, 1, 2)

In [67]:
d['data'][:,:,:,auto_idxs,:,0].shape

(30, 144, 16, 2, 1)

In [68]:
np.array(packets)['data'][:,:,:,:,auto_idxs,0,0].shape

(6, 144, 16, 2, 30)

In [69]:
avg.calc_ics(packets, auto_idxs).shape

(6, 144, 16, 2)

In [70]:
%timeit avg.calc_ics(packets, auto_idxs).shape

3.41 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
from craco import card_averager

In [72]:

dummy_packet = packets[0].copy()
print(type(dummy_packet))

avger = card_averager.Averager(nbeam,nant,nc=24,nt=32,npol=1,vis_fscrunch=6, vis_tscrunch=4, dummy_packet=dummy_packet, cdtype=np.float32)


<class 'numpy.ndarray'>



LV: Checking a loop in "_ZN5craco13card_averager9average15B3v21B60c8tJTIeFIjxB2IKSgI4CrvQClYb5wBbdC9XqICn1Wk1gKBZBVGsCAA_3d_3dE625ListType_5bunaligned_20array_28Record_28frame_id_5btype_3duint64_3boffset_3d0_5d_2cbat_5btype_3duint64_3boffset_3d8_5d_2cbeam_number_5btype_3duint8_3boffset_3d16_5d_2csample_number_5btype_3duint8_3boffset_3d17_5d_2cchannel_number_5btype_3duint8_3boffset_3d18_5d_2cfpga_id_5btype_3duint8_3boffset_3d19_5d_2cnprod_5btype_3duint16_3boffset_3d20_5d_2cflags_5btype_3duint8_3boffset_3d22_5d_2czero1_5btype_3duint8_3boffset_3d23_5d_2cversion_5btype_3duint32_3boffset_3d24_5d_2czero3_5btype_3duint32_3boffset_3d28_5d_2cdata_5btype_3dnestedarray_28int16_2c_20_282_2c_20930_29_29_3boffset_3d32_5d_3b3752_3bFalse_29_2c_202d_2c_20C_29_5d5ArrayIbLi1E1C7mutable7alignedE42reflected_20list_28int64_29_3civ_3dNone_3e5ArrayI6RecordILi2225EELi1E1C7mutable9unalignedEx5ArrayIfLi3E1C7mutable7alignedE" from average15
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B316.us.3

In [73]:
valid = np.ones(6, dtype=bool)

In [74]:
%timeit avg.calc_and_reshape_ics(packets, auto_idxs, valid, avger.output['ics'])

3.24 ms ± 237 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:
# current averager takes 270 milliseconds - which is just a bloody outrage - I mean seriously
%timeit avger.accumulate_packets(packets)

31.2 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [76]:
avger = card_averager.Averager(nbeam,nant,24,32,1,6,4, dummy_packet=dummy_packet, cdtype=np.float32)

In [77]:
avger.output.dtype

dtype([('ics', '<f4', (32, 24)), ('cas', '<f4', (32, 24)), ('vis', '<f4', (435, 4, 8, 2))])

In [78]:
avger.output.shape

(36,)

In [79]:
avger.output['vis'].shape

(36, 435, 4, 8, 2)

In [80]:
from craco.card_averager import ibc2beamchan


_= average7(din, tscrunch=8, dout=avger.output['vis'])
%timeit average7(din, tscrunch=8, dout=avger.output['vis'])


LV: Checking a loop in "_ZN5craco13card_averager8average7B3v32B66c8tJTC_2fWwGaMLSgd6CjA4xXKR0tsqKIYSQsqDfOsLbpvq9VBUuq1msAAL4Ko1gQAE5ArrayI6RecordILi2031EELi3E1C7mutable9unalignedEx5ArrayIfLi5E1A7mutable7alignedE" from card_averager.py:335:1
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: .766.endif
LV: Found an induction variable.
LV: We can vectorize this loop!
LV: Found trip count: 0
LV: The max safe fixed VF is: 67108864.
LV: The max safe scalable VF is: vscale x 0.
LV: The Smallest and Widest types: 32 / 32 bits.
LV: The Widest register safe to use is: 512 bits.
LV: The Widest register safe to use is: vscale x 0 bits.
LV: The target has no scalable vector registers.
LV: Found uniform instruction:   %exitcond1432.not = icmp eq i64 %.839, %arg.dout.5.4, !dbg !98
LV: Found uniform instruction:   %exitcond1432.not = icmp eq i64 %.839, %arg.dout.5.4, !dbg !98
LV: Found uniform instruction:   %exitcond1432.not = icmp eq i64 %.839, %arg.dout.5.4, !dbg !98
LV: Found unifor

45.5 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [81]:


_= average8(din, tscrunch=8, dout=avger.output['vis'],nant=nant)
%timeit average8(din, tscrunch=8, dout=avger.output['vis'],nant=nant)


LV: Checking a loop in "_ZN5craco13card_averager8average8B3v34B66c8tJTC_2fWwGaMLSgd6CjA4xXKR0tsqKIYSQsqDfOsLbpvq9VBUuq1msAAL4Ko1gQAE5ArrayI6RecordILi2031EELi3E1C7mutable9unalignedEx5ArrayIfLi5E1A7mutable7alignedEx" from card_averager.py:370:1
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: .770.endif
LV: Found an induction variable.
LV: We can vectorize this loop!
LV: Found trip count: 0
LV: The max safe fixed VF is: 67108864.
LV: The max safe scalable VF is: vscale x 0.
LV: The Smallest and Widest types: 32 / 32 bits.
LV: The Widest register safe to use is: 512 bits.
LV: The Widest register safe to use is: vscale x 0 bits.
LV: The target has no scalable vector registers.
LV: Found uniform instruction:   %exitcond2349.not = icmp eq i64 %.843, %arg.dout.5.4, !dbg !110
LV: Found uniform instruction:   %exitcond2349.not = icmp eq i64 %.843, %arg.dout.5.4, !dbg !110
LV: Found uniform instruction:   %exitcond2349.not = icmp eq i64 %.843, %arg.dout.5.4, !dbg !110
LV: Found un

43.7 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [82]:
avger.output['vis'].shape

(36, 435, 4, 8, 2)

In [83]:
average2(din,tscrunch=8).shape

(144, 4, 465, 2)

In [84]:
def average_vis_and_reshape2(din, tscrunch, dout, auto_idxs, cross_idxs):
    '''
    Writes (beam,chan) order and removes autocorrelations
    Fxed fscrunch at 6
    This is horriffically slow in NUMPY - really, really bad
    '''
    data = din['data']
    (nfpga, npkt, nt1, nt2, nblall, _, _) = data.shape
    dout[:] = 0
    # HACK - set nbl to output NBL, which doesn't include autos, for now
    #nbl = dout.shape[1]
    ntout = nt1*nt2//tscrunch
    
    nbl = len(cross_idxs)
    nant = len(auto_idxs)
    dcross = data[:,:,:,:,cross_idxs,:,:]
    d = dcross.reshape(nfpga, npkt, ntout, tscrunch, nbl, 2).mean(axis=(0,3), dtype=np.float32)
    dcopy1 = d[:32*4,...].reshape(32,4,ntout,nbl,2).transpose(0,3,1,2,4)
    dcopy2 = d[32*4:,...].reshape(4,4,ntout,nbl,2).transpose(0,3,1,2,4)


    dout[0:32, :,:,:,:] = dcopy1
    dout[32:, :,:,:,:] = dcopy2

            
    return dout

print(average_vis_and_reshape2(din, tscrunch=4, dout=avger.output['vis'],auto_idxs=auto_idxs,cross_idxs=cross_idxs).shape)
%timeit average_vis_and_reshape2(din, tscrunch=4, dout=avger.output['vis'],auto_idxs=auto_idxs,cross_idxs=cross_idxs)

(36, 435, 4, 8, 2)
819 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
avger.output['vis'].shape

(36, 435, 4, 8, 2)

In [86]:
avger.output['cas'].shape

(36, 32, 24)

In [87]:
din['data'].shape

(6, 144, 16, 2, 465, 1, 2)

In [88]:
data = din['data'].astype(np.float32)
data.shape

(6, 144, 16, 2, 465, 1, 2)

In [89]:
def amp1(data):
    return np.sqrt(data[...,0]**2 + data[...,1]**2)

%timeit amp1(data)

40.9 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [90]:
from math import sqrt

@njit
def amp2(data, dout):
    (nfpga, npkt, nt1, nt2, nbl, _, _) = data.shape
    dout[:] = 0
    # HACK - set nbl to output NBL, which doesn't include autos, for now
    nbl = dout.shape[1]
    
    # dout shape
    # avger.output['vis'].shape
    # (36, 435, 4, 8, 2)
    
    for ifpga in range(nfpga):
        for ipkt in range(npkt):
            beam,chan = ibc2beamchan(ipkt)
            #print(beam,chan)
            for t1 in range(nt1):
                for t2 in range(nt2):
                    ttotal = t2 + t1*nt2
                    tout = ttotal // tscrunch
                    for ibl in range(nbl):
                        d0 = data[ifpga,ipkt,t1,t2,ibl,0,0]
                        d1 = data[ifpga,ipkt,t1,t2,ibl,0,1]

                        da = sqrt(d0**2 + d1**2)    
                        dout[ifpga,ipkt,t1,t2,ibl,0] = da
                        
    return dout
                            
    

In [91]:
dout = np.zeros(data.shape[:-1], dtype=np.float32)
amp2(data, dout).shape
%timeit amp2(data, dout)


LV: Checking a loop in "_ZN5numba7cpython7numbers14int_power_impl12_3clocals_3e9int_powerB3v36B42c8tJTC_2fWQA93W1AaAIYBPIqRBFCjDSZRVAJmaQIAEfx" from int_power
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B138
LV: Not vectorizing: Found an unidentified PHI   %a.1.2.0 = phi float [ %.149, %B138 ], [ %arg.a, %B138.preheader ]
LV: Interleaving disabled by the pass manager
LV: Can't vectorize the instructions or CFG
LV: Not vectorizing: Cannot prove legality.

LV: Checking a loop in "_ZN8__main__4amp2B3v35B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dE5ArrayIfLi7E1C7mutable7alignedE5ArrayIfLi6E1C7mutable7alignedE" from amp2
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: .594.endif
LV: Not vectorizing: Control flow cannot be substituted for a select   br label %.618.if
LV: Interleaving disabled by the pass manager
LV: Can't if-convert the loop.
LV: Not vectorizing: Cannot prove legality.

LV: Checking a loop in "_ZN8__main__4amp2B3v35B38c8tJTIeFIjxB2IKSgI4CrvQCl

18.3 ms ± 335 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
data.shape

(6, 144, 16, 2, 465, 1, 2)

In [93]:
avger.output['ics'].shape

(36, 32, 24)

In [94]:
%timeit calc_ics(data, auto_idxs)

7.5 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [95]:
dmean = calc_ics(data, auto_idxs)
dmean.shape

(6, 144, 16, 2)

In [96]:
icsout = avger.output['ics']

In [97]:
icsout.shape

(36, 32, 24)

In [98]:
(nfpga, npkt, nt1, nt2, nbl, npol, _) = data.shape
data.shape

(6, 144, 16, 2, 465, 1, 2)

In [99]:
dmean[:,:32*4,:,:].reshape(nfpga,4,32,32).transpose(2,3,1,0).reshape(32,32,24).shape

(32, 32, 24)

In [100]:
dmean[:,32*4:,:,:].reshape(nfpga,4,4,32).transpose(2,3,1,0).reshape(4,32,24).shape

(4, 32, 24)

In [101]:
average

<function average at 0x7f4f740d7c30>

# An easy thing for numba

In [102]:
@njit
def numba_sum(v):
    x = 0
    for i in range(len(v)):
        x += v[i]

    return x

In [103]:
v = np.arange(1024)
numba_sum(v)


LV: Checking a loop in "_ZN8__main__9numba_sumB3v37B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dE5ArrayIxLi1E1C7mutable7alignedE" from numba_sum
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B20
LV: Found an induction variable.
LV: We can vectorize this loop!
LV: Found trip count: 0
LV: The max safe fixed VF is: 33554432.
LV: The max safe scalable VF is: vscale x 0.
LV: The Smallest and Widest types: 64 / 64 bits.
LV: The Widest register safe to use is: 512 bits.
LV: The Widest register safe to use is: vscale x 0 bits.
LV: The target has no scalable vector registers.
LV: Found uniform instruction:   %exitcond.not = icmp eq i64 %.125, %spec.select
LV: Found uniform instruction:   %.178 = getelementptr i64, i64* %arg.v.4, i64 %.54.03
LV: Found uniform instruction:   %.54.03 = phi i64 [ %.125, %B20 ], [ 0, %B20.preheader ]
LV: Found uniform instruction:   %.125 = add nuw nsw i64 %.54.03, 1
LV: Found scalar instruction:   %.54.03 = phi i64 [ %.125, %B20 ], [ 0, %B20.preheader

523776

In [104]:
%timeit numba_sum(v)

350 ns ± 4.58 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [105]:
%timeit v.sum()

1.62 µs ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [106]:
packet_list[0].dtype

dtype([('frame_id', '<u8'), ('bat', '<u8'), ('beam_number', 'u1'), ('sample_number', 'u1'), ('channel_number', 'u1'), ('fpga_id', 'u1'), ('nprod', '<u2'), ('flags', 'u1'), ('zero1', 'u1'), ('version', '<u4'), ('zero3', '<u4'), ('data', '<i2', (2, 465, 1, 2))])

In [107]:
dout.dtype

dtype('float32')

In [108]:
dout1.shape

(144, 8, 465, 2)

In [109]:
dout2.shape

(144, 16, 2, 465, 1, 2)


# How about we try this all again

In [110]:
# setup for 3.7ms mode 
print(nbeam, nant, dummy_packet.shape)
avger = card_averager.Averager(nbeam,nant,nc=24,nt=32,npol=1,vis_fscrunch=6, vis_tscrunch=1, dummy_packet=dummy_packet, cdtype=np.float32)


36 30 (144, 16)



LV: Checking a loop in "_ZN5craco13card_averager9average15B3v38B60c8tJTIeFIjxB2IKSgI4CrvQClYb5wBbdC9XqICn1Wk1gKBZBVGsCAA_3d_3dE625ListType_5bunaligned_20array_28Record_28frame_id_5btype_3duint64_3boffset_3d0_5d_2cbat_5btype_3duint64_3boffset_3d8_5d_2cbeam_number_5btype_3duint8_3boffset_3d16_5d_2csample_number_5btype_3duint8_3boffset_3d17_5d_2cchannel_number_5btype_3duint8_3boffset_3d18_5d_2cfpga_id_5btype_3duint8_3boffset_3d19_5d_2cnprod_5btype_3duint16_3boffset_3d20_5d_2cflags_5btype_3duint8_3boffset_3d22_5d_2czero1_5btype_3duint8_3boffset_3d23_5d_2cversion_5btype_3duint32_3boffset_3d24_5d_2czero3_5btype_3duint32_3boffset_3d28_5d_2cdata_5btype_3dnestedarray_28int16_2c_20_282_2c_20930_29_29_3boffset_3d32_5d_3b3752_3bFalse_29_2c_202d_2c_20C_29_5d5ArrayIbLi1E1C7mutable7alignedE42reflected_20list_28int64_29_3civ_3dNone_3e5ArrayI6RecordILi2514EELi1E1C7mutable9unalignedEx5ArrayIfLi3E1C7mutable7alignedE" from average15
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B316.us.3

In [111]:
avger.dtype

dtype([('ics', '<f4', (32, 24)), ('cas', '<f4', (32, 24)), ('vis', '<f4', (435, 4, 32, 2))])

In [112]:
avger.output.dtype

dtype([('ics', '<f4', (32, 24)), ('cas', '<f4', (32, 24)), ('vis', '<f4', (435, 4, 32, 2))])

In [113]:
avger.output.shape

(36,)

In [114]:
valid = np.ones(6, dtype=bool)
valid

array([ True,  True,  True,  True,  True,  True])

In [115]:
def myclear(packets, valid, output):
    output['vis'] = 0
    output['ics'] = 0
    

In [116]:
%timeit myclear(packets, valid, avger.output)

1.09 ms ± 49.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [117]:
packet_list[0]['data'].shape

(144, 16, 2, 465, 1, 2)

In [118]:
packet_list[0].dtype

dtype([('frame_id', '<u8'), ('bat', '<u8'), ('beam_number', 'u1'), ('sample_number', 'u1'), ('channel_number', 'u1'), ('fpga_id', 'u1'), ('nprod', '<u2'), ('flags', 'u1'), ('zero1', 'u1'), ('version', '<u4'), ('zero3', '<u4'), ('data', '<i2', (2, 465, 1, 2))])

In [119]:
# average all frequency channels together - no time averaging
def average14(packets, valid, auto_idxs, cross_idxs, output, tscrunch=1, scratch=None):
    '''
    Fscrunch by 6
    '''
    nfpga = len(packets)
    vis = output['vis']
    #print(packets[0]['data'].shape, vis.shape)
    nbeam,ncross, nchan, nt, ncomp = vis.shape
    _,nt1,nt2,nbl,npol,_ = packets[0]['data'].shape
    ncout = NCHAN
    ntout = nt1*nt2 // tscrunch

    if scratch is None:
        scratch = np.zeros((ncout, ntout, nbl, 2), dtype=np.float32)
            
    for ibeam in range(NBEAM):
        scratch[:] = 0
        for ichan in range(ncout):
            for ifpga in range(nfpga):
                for it1 in range(nt1):
                    for it2 in range(nt2):
                        ibc = beamchan2ibc(ibeam, ichan)
                        tidx = it2 + nt2*it1
                        toutidx = tidx // tscrunch
                        scratch[ichan, toutidx, :,:] += packets[ifpga][ibc,it1]['data'][it2,:,0,:]

    return (output, scratch)
        

In [120]:
(output, scratch) = average14(packet_list, valid, auto_idxs, cross_idxs, avger.output, avger.vis_tscrunch)
#scratch = np.zeros((NCHAN, 64, nbl, 2), dtype=np.float32)
(output, scratch) = average14(packet_list, valid, auto_idxs, cross_idxs, avger.output, avger.vis_tscrunch, scratch)

In [121]:
%timeit average14(packet_list, valid, auto_idxs, cross_idxs, avger.output, avger.vis_tscrunch)

256 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [122]:
# unbelievably, supplying pre-allocated scratch makes it 5 x slower! to 150 ms!?
%timeit average14(packet_list, valid, auto_idxs, cross_idxs, avger.output, avger.vis_tscrunch, scratch)

235 ms ± 7.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [123]:
packets[0].dtype

dtype([('frame_id', '<u8'), ('bat', '<u8'), ('beam_number', 'u1'), ('sample_number', 'u1'), ('channel_number', 'u1'), ('fpga_id', 'u1'), ('nprod', '<u2'), ('flags', 'u1'), ('zero1', 'u1'), ('version', '<u4'), ('zero3', '<u4'), ('data', '<i2', (2, 465, 1, 2))])

In [124]:
packets[0].dtype['data'].shape

(2, 465, 1, 2)

In [125]:
from craco import cardcapfile

In [126]:
flat_dtype = cardcapfile.debughdr[:]
flat_dtype.append(('data','<i2', (2, nbl*2)))
flat_dtype = np.dtype(flat_dtype)
flat_dtype

dtype([('frame_id', '<u8'), ('bat', '<u8'), ('beam_number', 'u1'), ('sample_number', 'u1'), ('channel_number', 'u1'), ('fpga_id', 'u1'), ('nprod', '<u2'), ('flags', 'u1'), ('zero1', 'u1'), ('version', '<u4'), ('zero3', '<u4'), ('data', '<i2', (2, 930))])

In [127]:
packet_list_flat = List()
[packet_list_flat.append(pkt.view(flat_dtype)) for pkt in packet_list]

[None, None, None, None, None, None]

In [128]:
packet_list_flat[0].dtype

dtype([('frame_id', '<u8'), ('bat', '<u8'), ('beam_number', 'u1'), ('sample_number', 'u1'), ('channel_number', 'u1'), ('fpga_id', 'u1'), ('nprod', '<u2'), ('flags', 'u1'), ('zero1', 'u1'), ('version', '<u4'), ('zero3', '<u4'), ('data', '<i2', (2, 930))])

In [131]:
_,nt1,nt2, nprod = packet_list_flat[0]['data'].shape
ntout = nt1*nt2//avger.vis_tscrunch
ncout = NCHAN
scratch = np.zeros((ncout, ntout, nprod), dtype=np.float32)
(output, scratch) = average15(packet_list_flat, valid, cross_idxs, avger.output, avger.vis_tscrunch, scratch)
#scratch = np.zeros((NCHAN, 64, nbl, 2), dtype=np.float32)
#(output, scratch) = average15(packet_list_flat, valid, auto_idxs, cross_idxs, avger.output, avger.vis_tscrunch, scratch)


LV: Checking a loop in "_ZN5craco13card_averager9average15B3v42B60c8tJTIeFIjxB2IKSgI4CrvQClYb5wBbdC9XqICn1Wk1gKBZBVGsCAA_3d_3dE625ListType_5bunaligned_20array_28Record_28frame_id_5btype_3duint64_3boffset_3d0_5d_2cbat_5btype_3duint64_3boffset_3d8_5d_2cbeam_number_5btype_3duint8_3boffset_3d16_5d_2csample_number_5btype_3duint8_3boffset_3d17_5d_2cchannel_number_5btype_3duint8_3boffset_3d18_5d_2cfpga_id_5btype_3duint8_3boffset_3d19_5d_2cnprod_5btype_3duint16_3boffset_3d20_5d_2cflags_5btype_3duint8_3boffset_3d22_5d_2czero1_5btype_3duint8_3boffset_3d23_5d_2cversion_5btype_3duint32_3boffset_3d24_5d_2czero3_5btype_3duint32_3boffset_3d28_5d_2cdata_5btype_3dnestedarray_28int16_2c_20_282_2c_20930_29_29_3boffset_3d32_5d_3b3752_3bFalse_29_2c_202d_2c_20C_29_5d5ArrayIbLi1E1C7mutable7alignedE42reflected_20list_28int64_29_3civ_3dNone_3e5ArrayI6RecordILi2514EELi1E1C7mutable9unalignedEx5ArrayIfLi3E1C7mutable7alignedE" from average15
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B316.us.3

In [132]:
%timeit (output, scratch2) = average15(packet_list_flat, valid, cross_idxs, avger.output, avger.vis_tscrunch, scratch)


28.7 ms ± 967 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [133]:
nops = len(packet_list_flat)*packet_list_flat[0]['data'].size
nops


25712640

In [134]:
nops_required = nops/0.110
nops_required/1e9

0.23375127272727272

In [135]:

@njit
def calc_ics1(packets_flat, valid, auto_idxs, output):
    '''
    Better version of calc_ics = does all the good stuff in 1ms.
    '''
    nfpga = len(packets_flat)
    vis = output['vis']
    #print(packets_flat[0]['data'].shape, vis.shape)
    nbeam,ncross, nchan, nt, ncomp = vis.shape
    _,nt1,nt2, nprod = packets_flat[0]['data'].shape
    nant = len(auto_idxs)
    scale = np.float32(1/nant)
    
    for ibeam in prange(NBEAM):
        # calculate ICS
        output[ibeam]['ics'][:] = 0

        for ic in range(NCHAN):
            ibc = beamchan2ibc(ibeam, ic)
            for ifpga in range(nfpga):
                cout =  ifpga + nfpga * ic                        
                for it1 in range(nt1):
                    d = packets_flat[ifpga][ibc,it1]['data']
                    for it2 in range(nt2):
                        s = np.float32(0)
                        tout = it2 + nt2*it1
                        for ibl in range(len(auto_idxs)):
                            blidx = auto_idxs[ibl]                            
                            idx = blidx*2 # find real part only of value were every second part is real
                            s += d[it2,idx]
                        
                        output[ibeam]['ics'][tout, cout] = s*scale
    return output

In [136]:
_ = calc_ics1(packet_list_flat, valid, auto_idxs, output)



LV: Checking a loop in "_ZN8__main__9calc_ics1B3v44B38c8tJTIeFIjxB2IKSgI4CrvQClQZ6FczSBAA_3dE625ListType_5bunaligned_20array_28Record_28frame_id_5btype_3duint64_3boffset_3d0_5d_2cbat_5btype_3duint64_3boffset_3d8_5d_2cbeam_number_5btype_3duint8_3boffset_3d16_5d_2csample_number_5btype_3duint8_3boffset_3d17_5d_2cchannel_number_5btype_3duint8_3boffset_3d18_5d_2cfpga_id_5btype_3duint8_3boffset_3d19_5d_2cnprod_5btype_3duint16_3boffset_3d20_5d_2cflags_5btype_3duint8_3boffset_3d22_5d_2czero1_5btype_3duint8_3boffset_3d23_5d_2cversion_5btype_3duint32_3boffset_3d24_5d_2czero3_5btype_3duint32_3boffset_3d28_5d_2cdata_5btype_3dnestedarray_28int16_2c_20_282_2c_20930_29_29_3boffset_3d32_5d_3b3752_3bFalse_29_2c_202d_2c_20C_29_5d5ArrayIbLi1E1C7mutable7alignedE42reflected_20list_28int64_29_3civ_3dNone_3e5ArrayI6RecordILi2514EELi1E1C7mutable9unalignedE" from calc_ics1
LV: Loop hints: force=? width=0 interleave=0
LV: Found a loop: B236.1
LV: Found an induction variable.
LV: Found FP op with unsafe algebra

In [137]:
%timeit calc_ics1(packet_list_flat, valid, auto_idxs, output)


1.9 ms ± 78.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [141]:
%timeit avger.accumulate_packets(packets)

30.7 ms ± 338 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
